In [2]:
import sys
sys.path.append('../../')
from MPRA_predict.utils import *
os.chdir('../../data/SirajMPRA')

from Bio import pairwise2
from Bio.pairwise2 import format_alignment, align

/home/hxcai/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df4 = pd.read_excel('supp/media-4.xlsx')
df4

,Supplementary Table 3 | MPRA Results; best measurement across libraries (by Plasmid coverage across variants) chosen per variant per cell type,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
0,variant,active.any,emVar.any,allele1.log2FC_A549,allele1.log2FC_HEPG2,allele1.log2FC_K562,allele1.log2FC_SKNSH,allele1.log2FC_HCT116,allele1.log2FC.SE_A549,allele1.log2FC.SE_HEPG2,...,mean_RNA_ref_A549,mean_RNA_ref_HEPG2,mean_RNA_ref_K562,mean_RNA_ref_SKNSH,mean_RNA_ref_HCT116,mean_RNA_alt_A549,mean_RNA_alt_HEPG2,mean_RNA_alt_K562,mean_RNA_alt_SKNSH,mean_RNA_alt_HCT116
1,chr10:100025816:G:A,True,False,NaN,1.623685,2.044789,1.22237,1.280575,NaN,0.155521,...,NaN,453,593.166667,305.4,459,NaN,719.6,796,599.4,656.2
2,chr10:100029561:C:T,True,True,NaN,3.390524,0.296395,3.402503,2.162766,NaN,0.088808,...,NaN,6677.2,665.166667,5250,3554.4,NaN,2298.8,439.333333,1873.8,1458.4
3,chr10:100098716:G:T,True,True,NaN,1.174309,0.853279,1.283911,1.000218,NaN,0.071422,...,NaN,3417.4,1131.666667,2423,2850,NaN,1243.4,374.333333,1114.2,1060.4
4,chr10:100109592:T:C,False,False,NaN,-0.007378,0.301644,0.592409,-0.03329,NaN,0.139805,...,NaN,564.6,601.666667,643.4,718.6,NaN,695.2,522.5,802.6,754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304274,chrX:99995091:A:G,False,False,NaN,0.171016,0.41844,-0.541239,0.296166,NaN,0.136126,...,NaN,576.2,390.166667,281,794,NaN,211.6,226,140.2,320.4
304275,chrX:99995933:C:G,False,False,NaN,0.433557,0.23761,0.368939,-0.133435,NaN,0.064148,...,NaN,1524.6,562.666667,884.2,948.6,NaN,1853.8,657.833333,1382.6,1364.2
304276,chrX:99996102:A:G,False,False,NaN,0.09405,0.310849,0.070434,0.048445,NaN,0.060897,...,NaN,1862,953.333333,1182,1679.4,NaN,1936,908.166667,1137.8,1626.6
304277,chrX:99998829:C:T,False,False,NaN,-0.496841,0.125072,0.011764,-0.355709,NaN,0.132722,...,NaN,259.8,123.666667,207.4,265,NaN,306.4,118.166667,206.8,402.6


In [4]:
df4.columns = df4.iloc[0]
df4 = df4.iloc[1:].reset_index(drop=True)
df4 = df4.rename(columns={'variant': 'Variant'})
df4

,Variant,active.any,emVar.any,allele1.log2FC_A549,allele1.log2FC_HEPG2,allele1.log2FC_K562,allele1.log2FC_SKNSH,allele1.log2FC_HCT116,allele1.log2FC.SE_A549,allele1.log2FC.SE_HEPG2,...,mean_RNA_ref_A549,mean_RNA_ref_HEPG2,mean_RNA_ref_K562,mean_RNA_ref_SKNSH,mean_RNA_ref_HCT116,mean_RNA_alt_A549,mean_RNA_alt_HEPG2,mean_RNA_alt_K562,mean_RNA_alt_SKNSH,mean_RNA_alt_HCT116
0,chr10:100025816:G:A,True,False,NaN,1.623685,2.044789,1.22237,1.280575,NaN,0.155521,...,NaN,453,593.166667,305.4,459,NaN,719.6,796,599.4,656.2
1,chr10:100029561:C:T,True,True,NaN,3.390524,0.296395,3.402503,2.162766,NaN,0.088808,...,NaN,6677.2,665.166667,5250,3554.4,NaN,2298.8,439.333333,1873.8,1458.4
2,chr10:100098716:G:T,True,True,NaN,1.174309,0.853279,1.283911,1.000218,NaN,0.071422,...,NaN,3417.4,1131.666667,2423,2850,NaN,1243.4,374.333333,1114.2,1060.4
3,chr10:100109592:T:C,False,False,NaN,-0.007378,0.301644,0.592409,-0.03329,NaN,0.139805,...,NaN,564.6,601.666667,643.4,718.6,NaN,695.2,522.5,802.6,754
4,chr10:100141027:T:C,False,False,NaN,-0.238128,-0.00355,-0.288184,0.041701,NaN,0.076976,...,NaN,645.4,361.666667,417,731.2,NaN,627.8,374.666667,333.2,660.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304273,chrX:99995091:A:G,False,False,NaN,0.171016,0.41844,-0.541239,0.296166,NaN,0.136126,...,NaN,576.2,390.166667,281,794,NaN,211.6,226,140.2,320.4
304274,chrX:99995933:C:G,False,False,NaN,0.433557,0.23761,0.368939,-0.133435,NaN,0.064148,...,NaN,1524.6,562.666667,884.2,948.6,NaN,1853.8,657.833333,1382.6,1364.2
304275,chrX:99996102:A:G,False,False,NaN,0.09405,0.310849,0.070434,0.048445,NaN,0.060897,...,NaN,1862,953.333333,1182,1679.4,NaN,1936,908.166667,1137.8,1626.6
304276,chrX:99998829:C:T,False,False,NaN,-0.496841,0.125072,0.011764,-0.355709,NaN,0.132722,...,NaN,259.8,123.666667,207.4,265,NaN,306.4,118.166667,206.8,402.6


In [11]:
df4.columns

Index(['Variant', 'active.any', 'emVar.any', 'allele1.log2FC_A549',
       'allele1.log2FC_HEPG2', 'allele1.log2FC_K562', 'allele1.log2FC_SKNSH',
       'allele1.log2FC_HCT116', 'allele1.log2FC.SE_A549',
       'allele1.log2FC.SE_HEPG2', 'allele1.log2FC.SE_K562',
       'allele1.log2FC.SE_SKNSH', 'allele1.log2FC.SE_HCT116',
       'allele2.log2FC_A549', 'allele2.log2FC_HEPG2', 'allele2.log2FC_K562',
       'allele2.log2FC_SKNSH', 'allele2.log2FC_HCT116',
       'allele2.log2FC.SE_A549', 'allele2.log2FC.SE_HEPG2',
       'allele2.log2FC.SE_K562', 'allele2.log2FC.SE_SKNSH',
       'allele2.log2FC.SE_HCT116', 'log2FC_A549', 'log2FC_HEPG2',
       'log2FC_K562', 'log2FC_SKNSH', 'log2FC_HCT116', 'active_A549',
       'active_HEPG2', 'active_K562', 'active_SKNSH', 'active_HCT116',
       'log2FC.logPadj_A549', 'log2FC.logPadj_HEPG2', 'log2FC.logPadj_K562',
       'log2FC.logPadj_SKNSH', 'log2FC.logPadj_HCT116', 'log2Skew_A549',
       'log2Skew_HEPG2', 'log2Skew_K562', 'log2Skew_SKNSH', 'log

In [48]:
df4['active_HEPG2']

0          True
1          True
2          True
3         False
4         False
          ...  
304273    False
304274    False
304275    False
304276    False
304277    False
Name: active_HEPG2, Length: 304278, dtype: object

In [40]:
a = df4['allele1.log2FC_HEPG2'].astype(float)
b = df4['allele2.log2FC_HEPG2'].astype(float)
print(pearson(a, b))

a = df4['allele1.log2FC_HEPG2'].astype(float)
b = df4['log2FC_HEPG2'].astype(float)
print(pearson(a, b))

a = df4['allele2.log2FC_HEPG2'].astype(float)
b = df4['log2FC_HEPG2'].astype(float)
print(pearson(a, b))

a = df4['allele1.log2FC_HEPG2'].astype(float) + df4['allele2.log2FC_HEPG2'].astype(float)
b = df4['log2FC_HEPG2'].astype(float)
print(pearson(a, b))

0.9334397179120867
0.9729501150705911
0.9715276123001267
0.9888336549728861


In [47]:
a = np.log2(df4['mean_RNA_ref_HEPG2'].astype(float) / df4['mean_Plasmid_ref_HEPG2'].astype(float))
b = df4['allele1.log2FC_HEPG2'].astype(float)
print(pearson(a, b))

a = np.log2(df4['mean_RNA_ref_HEPG2'].astype(float) / df4['mean_Plasmid_ref_HEPG2'].astype(float))
b = df4['allele2.log2FC_HEPG2'].astype(float)
print(pearson(a, b))


a = np.log2(df4['mean_RNA_alt_HEPG2'].astype(float) / df4['mean_Plasmid_alt_HEPG2'].astype(float))
b = df4['allele1.log2FC_HEPG2'].astype(float)
print(pearson(a, b))

a = np.log2(df4['mean_RNA_alt_HEPG2'].astype(float) / df4['mean_Plasmid_alt_HEPG2'].astype(float))
b = df4['allele2.log2FC_HEPG2'].astype(float)
print(pearson(a, b))

0.8874464744378183
0.8286805968686911
0.828408715597724
0.8867039110428847


In [42]:
a = np.log2((df4['mean_RNA_ref_K562'] / df4['mean_Plasmid_ref_K562']).astype(float))
b = df4['allele1.log2FC_K562'].astype(float)
print(pearson(a, b))

a = np.log2((df4['mean_RNA_ref_K562'] / df4['mean_Plasmid_ref_K562']).astype(float))
b = df4['allele2.log2FC_K562'].astype(float)
print(pearson(a, b))

a = np.log2((df4['mean_RNA_ref_K562'] / df4['mean_Plasmid_ref_K562']).astype(float))
b = df4['log2FC_K562'].astype(float)
print(pearson(a, b))

0.9659776405506766
0.8973480432296914
0.9375230566885555


In [5]:
df3 = pd.read_excel('supp/media-3.xlsx')
df3

,Supplementary Table 2 |Variant Oligo Sequences,Unnamed: 1,Unnamed: 2
0,Variant,Allele 1 Oligo,Allele 2 Oligo
1,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...
2,chr1:522224:G:A,TCCTGTGGTTCTCTGTTCTTTTCTCAGGTGTGTCCTTAACCTTGCA...,TCCTGTGGTTCTCTGTTCTTTTCTCAGGTGTGTCCTTAACCTTGCA...
3,chr1:729679:C:G,TCCATGCAGTGATGGTCACCTTCTCTAACAGGCATTTCAGAGTGAG...,TCCATGCAGTGATGGTCACCTTCTCTAACAGGCATTTCAGAGTGAG...
4,chr1:741397:A:G,TTAGTCAATTAGCCAAAAAGAATTGGCAGTTAATGTCAATTAATTG...,TTAGTCAATTAGCCAAAAAGAATTGGCAGTTAATGTCAATTAATTG...
...,...,...,...
365758,chr4:57975328:A:G,TTTGCCCACAAATAATTAGCTAGTTAGATTACTGTTTGAGAACCTA...,TTTGCCCACAAATAATTAGCTAGTTAGATTACTGTTTGAGAACCTA...
365759,chr20:36661369:C:A,GGTCAACAGACACATTGGGAAGGCTAAGTTAGTTCCCTCCACCTTT...,GGTCAACAGACACATTGGGAAGGCTAAGTTAGTTCCCTCCACCTTT...
365760,chr8:60683160:C:T,GGCCATGGAGTTTATTCAAATTGTATATCTAAACAATTTTAGTATT...,GGCCATGGAGTTTATTCAAATTGTATATCTAAACAATTTTAGTATT...
365761,chr7:31046388:G:C,CTCAGGGCTCCATCCCCGCCTGGCCCCATCACCCCAGGCCCTCTGG...,CTCAGGGCTCCATCCCCGCCTGGCCCCATCACCCCAGGCCCTCTGG...


In [6]:
df3.columns = df3.iloc[0]
df3 = df3.iloc[1:]
df3

,Variant,Allele 1 Oligo,Allele 2 Oligo
1,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...
2,chr1:522224:G:A,TCCTGTGGTTCTCTGTTCTTTTCTCAGGTGTGTCCTTAACCTTGCA...,TCCTGTGGTTCTCTGTTCTTTTCTCAGGTGTGTCCTTAACCTTGCA...
3,chr1:729679:C:G,TCCATGCAGTGATGGTCACCTTCTCTAACAGGCATTTCAGAGTGAG...,TCCATGCAGTGATGGTCACCTTCTCTAACAGGCATTTCAGAGTGAG...
4,chr1:741397:A:G,TTAGTCAATTAGCCAAAAAGAATTGGCAGTTAATGTCAATTAATTG...,TTAGTCAATTAGCCAAAAAGAATTGGCAGTTAATGTCAATTAATTG...
5,chr1:752566:G:A,AAAGAATTTTTGGCTAAGTTCCCAAAAACGATTGCAACAAAACAAA...,AAAGAATTTTTGGCTAAGTTCCCAAAAACGATTGCAACAAAACAAA...
...,...,...,...
365758,chr4:57975328:A:G,TTTGCCCACAAATAATTAGCTAGTTAGATTACTGTTTGAGAACCTA...,TTTGCCCACAAATAATTAGCTAGTTAGATTACTGTTTGAGAACCTA...
365759,chr20:36661369:C:A,GGTCAACAGACACATTGGGAAGGCTAAGTTAGTTCCCTCCACCTTT...,GGTCAACAGACACATTGGGAAGGCTAAGTTAGTTCCCTCCACCTTT...
365760,chr8:60683160:C:T,GGCCATGGAGTTTATTCAAATTGTATATCTAAACAATTTTAGTATT...,GGCCATGGAGTTTATTCAAATTGTATATCTAAACAATTTTAGTATT...
365761,chr7:31046388:G:C,CTCAGGGCTCCATCCCCGCCTGGCCCCATCACCCCAGGCCCTCTGG...,CTCAGGGCTCCATCCCCGCCTGGCCCCATCACCCCAGGCCCTCTGG...


In [7]:
merged_df = pd.merge(df4, df3, on='Variant', how='inner')
merged_df

,Variant,active.any,emVar.any,allele1.log2FC_A549,allele1.log2FC_HEPG2,allele1.log2FC_K562,allele1.log2FC_SKNSH,allele1.log2FC_HCT116,allele1.log2FC.SE_A549,allele1.log2FC.SE_HEPG2,...,mean_RNA_ref_K562,mean_RNA_ref_SKNSH,mean_RNA_ref_HCT116,mean_RNA_alt_A549,mean_RNA_alt_HEPG2,mean_RNA_alt_K562,mean_RNA_alt_SKNSH,mean_RNA_alt_HCT116,Allele 1 Oligo,Allele 2 Oligo
0,chr10:100025816:G:A,True,False,NaN,1.623685,2.044789,1.22237,1.280575,NaN,0.155521,...,593.166667,305.4,459,NaN,719.6,796,599.4,656.2,TCTGACCCCTCCCACGCTTCCTTTCCAGACTGAAACACTGTCTTCT...,TCTGACCCCTCCCACGCTTCCTTTCCAGACTGAAACACTGTCTTCT...
1,chr10:100029561:C:T,True,True,NaN,3.390524,0.296395,3.402503,2.162766,NaN,0.088808,...,665.166667,5250,3554.4,NaN,2298.8,439.333333,1873.8,1458.4,GCAACACAGAGGGTTCGGAAACGTGGGCTCTGCCTGCTGCCAATCA...,GCAACACAGAGGGTTCGGAAACGTGGGCTCTGCCTGCTGCCAATCA...
2,chr10:100098716:G:T,True,True,NaN,1.174309,0.853279,1.283911,1.000218,NaN,0.071422,...,1131.666667,2423,2850,NaN,1243.4,374.333333,1114.2,1060.4,CCCCCAGCAAACACGGATCTGCTCTCTGTCACTATCGATTAGCTTG...,CCCCCAGCAAACACGGATCTGCTCTCTGTCACTATCGATTAGCTTG...
3,chr10:100109592:T:C,False,False,NaN,-0.007378,0.301644,0.592409,-0.03329,NaN,0.139805,...,601.666667,643.4,718.6,NaN,695.2,522.5,802.6,754,GATCTCAGGCATCCTCACTTCGTGCCGAGAGGAGGCTCTTAAACAT...,GATCTCAGGCATCCTCACTTCGTGCCGAGAGGAGGCTCTTAAACAT...
4,chr10:100141027:T:C,False,False,NaN,-0.238128,-0.00355,-0.288184,0.041701,NaN,0.076976,...,361.666667,417,731.2,NaN,627.8,374.666667,333.2,660.4,CTCCTTTACAGGGAACATTTGCCTGGAGAACAGCTGCACCCTTGTG...,CTCCTTTACAGGGAACATTTGCCTGGAGAACAGCTGCACCCTTGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304273,chrX:99995091:A:G,False,False,NaN,0.171016,0.41844,-0.541239,0.296166,NaN,0.136126,...,390.166667,281,794,NaN,211.6,226,140.2,320.4,TAATTTATTCAGGAAAAATCTGAGGTCACTAACATTTACTAGTGCT...,TAATTTATTCAGGAAAAATCTGAGGTCACTAACATTTACTAGTGCT...
304274,chrX:99995933:C:G,False,False,NaN,0.433557,0.23761,0.368939,-0.133435,NaN,0.064148,...,562.666667,884.2,948.6,NaN,1853.8,657.833333,1382.6,1364.2,AGAGAGACTGACATCTCACTCCCAGGCTGGAGGCTTGCTGCACTAC...,AGAGAGACTGACATCTCACTCCCAGGCTGGAGGCTTGCTGCACTAC...
304275,chrX:99996102:A:G,False,False,NaN,0.09405,0.310849,0.070434,0.048445,NaN,0.060897,...,953.333333,1182,1679.4,NaN,1936,908.166667,1137.8,1626.6,AAGCCTGGTTTAAATTGCTCATAATGGCAGAGCAAAGAAAGAGATC...,AAGCCTGGTTTAAATTGCTCATAATGGCAGAGCAAAGAAAGAGATC...
304276,chrX:99998829:C:T,False,False,NaN,-0.496841,0.125072,0.011764,-0.355709,NaN,0.132722,...,123.666667,207.4,265,NaN,306.4,118.166667,206.8,402.6,TTATTATCTCCACTTTACAGATGAGAAAACTGTGGCTCAGATAAGT...,TTATTATCTCCACTTTACAGATGAGAAAACTGTGGCTCAGATAAGT...


In [8]:
merged_df.to_csv("SirajMPRA_raw.csv", index=False)

In [7]:
df_raw = pd.read_csv('SirajMPRA_raw.csv')
df_raw

/tmp/ipykernel_39214/1316199709.py:1: DtypeWarning: Columns (28,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('SirajMPRA_raw.csv')


,Variant,active.any,emVar.any,allele1.log2FC_A549,allele1.log2FC_HEPG2,allele1.log2FC_K562,allele1.log2FC_SKNSH,allele1.log2FC_HCT116,allele1.log2FC.SE_A549,allele1.log2FC.SE_HEPG2,...,mean_RNA_ref_K562,mean_RNA_ref_SKNSH,mean_RNA_ref_HCT116,mean_RNA_alt_A549,mean_RNA_alt_HEPG2,mean_RNA_alt_K562,mean_RNA_alt_SKNSH,mean_RNA_alt_HCT116,Allele 1 Oligo,Allele 2 Oligo
0,chr10:100025816:G:A,True,False,NaN,1.623685,2.044789,1.222370,1.280575,NaN,0.155521,...,593.166667,305.4,459.0,NaN,719.6,796.000000,599.4,656.2,TCTGACCCCTCCCACGCTTCCTTTCCAGACTGAAACACTGTCTTCT...,TCTGACCCCTCCCACGCTTCCTTTCCAGACTGAAACACTGTCTTCT...
1,chr10:100029561:C:T,True,True,NaN,3.390524,0.296395,3.402503,2.162766,NaN,0.088808,...,665.166667,5250.0,3554.4,NaN,2298.8,439.333333,1873.8,1458.4,GCAACACAGAGGGTTCGGAAACGTGGGCTCTGCCTGCTGCCAATCA...,GCAACACAGAGGGTTCGGAAACGTGGGCTCTGCCTGCTGCCAATCA...
2,chr10:100098716:G:T,True,True,NaN,1.174309,0.853279,1.283911,1.000218,NaN,0.071422,...,1131.666667,2423.0,2850.0,NaN,1243.4,374.333333,1114.2,1060.4,CCCCCAGCAAACACGGATCTGCTCTCTGTCACTATCGATTAGCTTG...,CCCCCAGCAAACACGGATCTGCTCTCTGTCACTATCGATTAGCTTG...
3,chr10:100109592:T:C,False,False,NaN,-0.007378,0.301644,0.592409,-0.033290,NaN,0.139805,...,601.666667,643.4,718.6,NaN,695.2,522.500000,802.6,754.0,GATCTCAGGCATCCTCACTTCGTGCCGAGAGGAGGCTCTTAAACAT...,GATCTCAGGCATCCTCACTTCGTGCCGAGAGGAGGCTCTTAAACAT...
4,chr10:100141027:T:C,False,False,NaN,-0.238128,-0.003550,-0.288184,0.041701,NaN,0.076976,...,361.666667,417.0,731.2,NaN,627.8,374.666667,333.2,660.4,CTCCTTTACAGGGAACATTTGCCTGGAGAACAGCTGCACCCTTGTG...,CTCCTTTACAGGGAACATTTGCCTGGAGAACAGCTGCACCCTTGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304273,chrX:99995091:A:G,False,False,NaN,0.171016,0.418440,-0.541239,0.296166,NaN,0.136126,...,390.166667,281.0,794.0,NaN,211.6,226.000000,140.2,320.4,TAATTTATTCAGGAAAAATCTGAGGTCACTAACATTTACTAGTGCT...,TAATTTATTCAGGAAAAATCTGAGGTCACTAACATTTACTAGTGCT...
304274,chrX:99995933:C:G,False,False,NaN,0.433557,0.237610,0.368939,-0.133435,NaN,0.064148,...,562.666667,884.2,948.6,NaN,1853.8,657.833333,1382.6,1364.2,AGAGAGACTGACATCTCACTCCCAGGCTGGAGGCTTGCTGCACTAC...,AGAGAGACTGACATCTCACTCCCAGGCTGGAGGCTTGCTGCACTAC...
304275,chrX:99996102:A:G,False,False,NaN,0.094050,0.310849,0.070434,0.048445,NaN,0.060897,...,953.333333,1182.0,1679.4,NaN,1936.0,908.166667,1137.8,1626.6,AAGCCTGGTTTAAATTGCTCATAATGGCAGAGCAAAGAAAGAGATC...,AAGCCTGGTTTAAATTGCTCATAATGGCAGAGCAAAGAAAGAGATC...
304276,chrX:99998829:C:T,False,False,NaN,-0.496841,0.125072,0.011764,-0.355709,NaN,0.132722,...,123.666667,207.4,265.0,NaN,306.4,118.166667,206.8,402.6,TTATTATCTCCACTTTACAGATGAGAAAACTGTGGCTCAGATAAGT...,TTATTATCTCCACTTTACAGATGAGAAAACTGTGGCTCAGATAAGT...


In [8]:
# 只保留ref和alt都len=200的样本

df = df_raw[(df_raw['Allele 1 Oligo'].str.len() == 200) & (df_raw['Allele 2 Oligo'].str.len() == 200)]
df = df.reset_index(drop=True)
df

,Variant,active.any,emVar.any,allele1.log2FC_A549,allele1.log2FC_HEPG2,allele1.log2FC_K562,allele1.log2FC_SKNSH,allele1.log2FC_HCT116,allele1.log2FC.SE_A549,allele1.log2FC.SE_HEPG2,...,mean_RNA_ref_K562,mean_RNA_ref_SKNSH,mean_RNA_ref_HCT116,mean_RNA_alt_A549,mean_RNA_alt_HEPG2,mean_RNA_alt_K562,mean_RNA_alt_SKNSH,mean_RNA_alt_HCT116,Allele 1 Oligo,Allele 2 Oligo
0,chr10:100025816:G:A,True,False,NaN,1.623685,2.044789,1.222370,1.280575,NaN,0.155521,...,593.166667,305.4,459.0,NaN,719.6,796.000000,599.4,656.2,TCTGACCCCTCCCACGCTTCCTTTCCAGACTGAAACACTGTCTTCT...,TCTGACCCCTCCCACGCTTCCTTTCCAGACTGAAACACTGTCTTCT...
1,chr10:100029561:C:T,True,True,NaN,3.390524,0.296395,3.402503,2.162766,NaN,0.088808,...,665.166667,5250.0,3554.4,NaN,2298.8,439.333333,1873.8,1458.4,GCAACACAGAGGGTTCGGAAACGTGGGCTCTGCCTGCTGCCAATCA...,GCAACACAGAGGGTTCGGAAACGTGGGCTCTGCCTGCTGCCAATCA...
2,chr10:100098716:G:T,True,True,NaN,1.174309,0.853279,1.283911,1.000218,NaN,0.071422,...,1131.666667,2423.0,2850.0,NaN,1243.4,374.333333,1114.2,1060.4,CCCCCAGCAAACACGGATCTGCTCTCTGTCACTATCGATTAGCTTG...,CCCCCAGCAAACACGGATCTGCTCTCTGTCACTATCGATTAGCTTG...
3,chr10:100109592:T:C,False,False,NaN,-0.007378,0.301644,0.592409,-0.033290,NaN,0.139805,...,601.666667,643.4,718.6,NaN,695.2,522.500000,802.6,754.0,GATCTCAGGCATCCTCACTTCGTGCCGAGAGGAGGCTCTTAAACAT...,GATCTCAGGCATCCTCACTTCGTGCCGAGAGGAGGCTCTTAAACAT...
4,chr10:100141027:T:C,False,False,NaN,-0.238128,-0.003550,-0.288184,0.041701,NaN,0.076976,...,361.666667,417.0,731.2,NaN,627.8,374.666667,333.2,660.4,CTCCTTTACAGGGAACATTTGCCTGGAGAACAGCTGCACCCTTGTG...,CTCCTTTACAGGGAACATTTGCCTGGAGAACAGCTGCACCCTTGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281322,chrX:99995091:A:G,False,False,NaN,0.171016,0.418440,-0.541239,0.296166,NaN,0.136126,...,390.166667,281.0,794.0,NaN,211.6,226.000000,140.2,320.4,TAATTTATTCAGGAAAAATCTGAGGTCACTAACATTTACTAGTGCT...,TAATTTATTCAGGAAAAATCTGAGGTCACTAACATTTACTAGTGCT...
281323,chrX:99995933:C:G,False,False,NaN,0.433557,0.237610,0.368939,-0.133435,NaN,0.064148,...,562.666667,884.2,948.6,NaN,1853.8,657.833333,1382.6,1364.2,AGAGAGACTGACATCTCACTCCCAGGCTGGAGGCTTGCTGCACTAC...,AGAGAGACTGACATCTCACTCCCAGGCTGGAGGCTTGCTGCACTAC...
281324,chrX:99996102:A:G,False,False,NaN,0.094050,0.310849,0.070434,0.048445,NaN,0.060897,...,953.333333,1182.0,1679.4,NaN,1936.0,908.166667,1137.8,1626.6,AAGCCTGGTTTAAATTGCTCATAATGGCAGAGCAAAGAAAGAGATC...,AAGCCTGGTTTAAATTGCTCATAATGGCAGAGCAAAGAAAGAGATC...
281325,chrX:99998829:C:T,False,False,NaN,-0.496841,0.125072,0.011764,-0.355709,NaN,0.132722,...,123.666667,207.4,265.0,NaN,306.4,118.166667,206.8,402.6,TTATTATCTCCACTTTACAGATGAGAAAACTGTGGCTCAGATAAGT...,TTATTATCTCCACTTTACAGATGAGAAAACTGTGGCTCAGATAAGT...


In [10]:
df_allele_1 = df[['Variant', 'Allele 1 Oligo', 'allele1.log2FC_A549', 'allele1.log2FC_HEPG2', 'allele1.log2FC_K562', 'allele1.log2FC_SKNSH', 'allele1.log2FC_HCT116']]
df_allele_2 = df[['Variant', 'Allele 2 Oligo', 'allele2.log2FC_A549', 'allele2.log2FC_HEPG2', 'allele2.log2FC_K562', 'allele2.log2FC_SKNSH', 'allele2.log2FC_HCT116']]
df_allele_1['is_ref'] = True
df_allele_2['is_ref'] = False

df_allele_1 = df_allele_1.rename(columns={'Variant': 'description', 'Allele 1 Oligo': 'seq', 'allele1.log2FC_A549': 'A549', 'allele1.log2FC_HEPG2': 'HepG2', 'allele1.log2FC_K562': 'K562', 'allele1.log2FC_SKNSH': 'SKNSH', 'allele1.log2FC_HCT116': 'HCT116'})
df_allele_2 = df_allele_2.rename(columns={'Variant': 'description', 'Allele 2 Oligo': 'seq', 'allele2.log2FC_A549': 'A549', 'allele2.log2FC_HEPG2': 'HepG2', 'allele2.log2FC_K562': 'K562', 'allele2.log2FC_SKNSH': 'SKNSH', 'allele2.log2FC_HCT116': 'HCT116'})

df = pd.concat([df_allele_1, df_allele_2], axis=0)
df = df.reset_index(drop=True)
df

/tmp/ipykernel_39214/1814766082.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_allele_1['is_ref'] = True
/tmp/ipykernel_39214/1814766082.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_allele_2['is_ref'] = False


,description,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref
0,chr10:100025816:G:A,TCTGACCCCTCCCACGCTTCCTTTCCAGACTGAAACACTGTCTTCT...,NaN,1.623685,2.044789,1.222370,1.280575,True
1,chr10:100029561:C:T,GCAACACAGAGGGTTCGGAAACGTGGGCTCTGCCTGCTGCCAATCA...,NaN,3.390524,0.296395,3.402503,2.162766,True
2,chr10:100098716:G:T,CCCCCAGCAAACACGGATCTGCTCTCTGTCACTATCGATTAGCTTG...,NaN,1.174309,0.853279,1.283911,1.000218,True
3,chr10:100109592:T:C,GATCTCAGGCATCCTCACTTCGTGCCGAGAGGAGGCTCTTAAACAT...,NaN,-0.007378,0.301644,0.592409,-0.033290,True
4,chr10:100141027:T:C,CTCCTTTACAGGGAACATTTGCCTGGAGAACAGCTGCACCCTTGTG...,NaN,-0.238128,-0.003550,-0.288184,0.041701,True
...,...,...,...,...,...,...,...,...
562649,chrX:99995091:A:G,TAATTTATTCAGGAAAAATCTGAGGTCACTAACATTTACTAGTGCT...,NaN,-0.188583,0.693828,-0.373362,0.203649,False
562650,chrX:99995933:C:G,AGAGAGACTGACATCTCACTCCCAGGCTGGAGGCTTGCTGCACTAC...,NaN,0.402471,-0.325169,0.395378,0.076269,False
562651,chrX:99996102:A:G,AAGCCTGGTTTAAATTGCTCATAATGGCAGAGCAAAGAAAGAGATC...,NaN,0.174034,0.108281,-0.039556,0.019372,False
562652,chrX:99998829:C:T,TTATTATCTCCACTTTACAGATGAGAAAACTGTGGCTCAGATAAGT...,NaN,-0.199757,0.231536,-0.198834,0.298371,False


In [12]:
df[['chr', 'pos', 'ref', 'alt']] = df['description'].str.split(':', expand=True)
df['pos'] = df['pos'].astype(int)
df['start'] = df['pos'] - 100
df['end'] = df['pos'] + 100
df = df.sort_values(by=['chr', 'pos']).reset_index(drop=True)
df

,description,seq,A549,HepG2,K562,SKNSH,HCT116,is_ref,chr,pos,ref,alt,start,end
0,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.667516,-0.704147,0.538641,0.159069,True,chr1,14677,G,A,14577,14777
1,chr1:14677:G:A,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,NaN,0.415984,0.554099,0.441904,-0.288019,False,chr1,14677,G,A,14577,14777
2,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.269937,-0.391222,0.752052,1.100437,True,chr1,63697,T,C,63597,63797
3,chr1:63697:T:C,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,NaN,1.151995,-0.659023,0.204721,1.050619,False,chr1,63697,T,C,63597,63797
4,chr1:64764:C:T,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,NaN,-0.211076,-0.007174,0.499376,0.257572,True,chr1,64764,C,T,64664,64864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562649,chrX:155233098:T:C,ATCATTTACTGGTGACTGCCCTGCTAGGGTGTCAGCTGTCAGATCC...,NaN,0.615976,1.927090,0.798263,1.035440,False,chrX,155233098,T,C,155232998,155233198
562650,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,1.409758,0.556338,2.843295,1.495515,True,chrX,155233794,A,G,155233694,155233894
562651,chrX:155233794:A:G,TGGTTACTGCAGGGGCAGGGTTTTGGCAGGAAATAAACATGCACGG...,NaN,0.894634,-0.072647,1.973661,0.794848,False,chrX,155233794,A,G,155233694,155233894
562652,chrX:155234730:T:C,TGGTGAGTTGCTTCTGTGCACACACTTTTGTTTATGAGTGTGCATG...,NaN,0.312172,0.165634,-0.041383,0.444472,True,chrX,155234730,T,C,155234630,155234830


In [13]:
df.to_csv("SirajMPRA_562654.csv", index=False)

In [93]:
info, seq1, seq2 = df.iloc[15]

print(info)
print(seq1)
print(seq2)

from pyfaidx import Fasta
genome = Fasta('/home/hxcai/genome/hg19.fa')
seq3 = genome['chr1'][1004389-100: 1004389+100].seq.upper()

print(seq3)
print(len(seq1), len(seq2), len(seq3))

alignments = align.globalms(seq1, seq2, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))

alignments = align.globalms(seq1, seq3, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))

alignments = align.globalms(seq2, seq3, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))

chr1:761957:A:AT
TTTCAATATACAATTAAGACTGAAAAGTATTGAGGCCTCATAAGCTGAACCTGACACAATAAATTTAAAAGGGAAACTAATTTGGAAATCAGAAAACCACTAAGGAATTTGGGAATTAGGCTTCTGCTGCCCTCTCTGCTACTGACGGTCAAGGCCTCCTCATTGTATTCTGTCCTCCATATCTCTGCTGATTCCCATT
TTTCAATATACAATTAAGACTGAAAAGTATTGAGGCCTCATAAGCTGAACCTGACACAATAAATTTAAAAGGGAAACTAATTTGGAAATCAGAAAACCATCTAAGGAATTTGGGAATTAGGCTTCTGCTGCCCTCTCTGCTACTGACGGTCAAGGCCTCCTCATTGTATTCTGTCCTCCATATCTCTGCTGATTCCCATT
TTTGTCCTCAGTGGGGGCATCGGATGCACGGGCACCATCATCGTGATTGACATCCTGGTGGACGTCATCCGCAGGCAGGGCGAGCCGCTCCACTCTGGGTCCCCCCGCCCTGCCCTGCTATAGCCCCACCCCTCCGGGCGACCCCACCCCTCCGGACGACCCCACCCCTCAGGACGACCCCACCCCTCAGGACGACCCCA
199 200 200
TTTCAATATACAATTAAGACTGAAAAGTATTGAGGCCTCATAAGCTGAACCTGACACAATAAATTTAAAAGGGAAACTAATTTGGAAATCAGAAAACCA-CTAAGGAATTTGGGAATTAGGCTTCTGCTGCCCTCTCTGCTACTGACGGTCAAGGCCTCCTCATTGTATTCTGTCCTCCATATCTCTGCTGATTCCCATT
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [78]:
info, seq1, seq2 = df.iloc[10]

print(info)
print(seq1)
print(seq2)

from pyfaidx import Fasta
genome = Fasta('/home/hxcai/genome/hg19.fa')
seq3 = genome['chr1'][759444-100+2: 759444+100+2].seq.upper()

print(seq3)
print(len(seq1), len(seq2), len(seq3))

alignments = align.globalms(seq1, seq2, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))

alignments = align.globalms(seq1, seq3, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))

alignments = align.globalms(seq2, seq3, 2, 0, -.5, -.1)
print(format_alignment(*alignments[0]))

chr1:759444:ATATT:A
GTGAGTATACAGTCAAAATAGTTGTGAAAAGAAAAAAATAAAGTTGTCAAATTTCATGTAACCTATGTTATTTGGAAATTTAAAATAAGTTATATAAATATTTATAACAATGTTAAACAATTTCATAAGTGGACTGAATAAAACATAATATTTTATCATATACAATAGCTCCTGCCTTCCTGAGGAAAACATAAAATCAA
GTGAGTATACAGTCAAAATAGTTGTGAAAAGAAAAAAATAAAGTTGTCAAATTTCATGTAACCTATGTTATTTGGAAATTTAAAATAAGTTATATAAATATAACAATGTTAAACAATTTCATAAGTGGACTGAATAAAACATAATATTTTATCATATACAATAGCTCCTGCCTTCCTGAGGAAAACATAAAATCAA
GTGAGTATACAGTCAAAATAGTTGTGAAAAGAAAAAAATAAAGTTGTCAAATTTCATGTAACCTATGTTATTTGGAAATTTAAAATAAGTTATATAAATATTTATAACAATGTTAAACAATTTCATAAGTGGACTGAATAAAACATAATATTTTATCATATACAATAGCTCCTGCCTTCCTGAGGAAAACATAAAATCAA
200 196 200
GTGAGTATACAGTCAAAATAGTTGTGAAAAGAAAAAAATAAAGTTGTCAAATTTCATGTAACCTATGTTATTTGGAAATTTAAAATAAGTTATATAAATATTTATAACAATGTTAAACAATTTCATAAGTGGACTGAATAAAACATAATATTTTATCATATACAATAGCTCCTGCCTTCCTGAGGAAAACATAAAATCAA
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||    ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||